# [o3]- Proyecto Ozono - Predictor_v0

# [0] - Inicialización

In [4]:
import findspark
findspark.init('/home/rulicering/BigData/spark-2.4.5-bin-hadoop2.7')
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import pandas as pd
from pyspark.sql.types import StructField,StringType,IntegerType,StructType,FloatType
import re as reg
import numpy as np
import datetime

In [5]:
spark = SparkSession.builder.appName('predictor').getOrCreate()

# [1] Datos

## [1.0] - Carga de ficheros (Datos, Predicción clima,  Calendario)

In [6]:
df_datos = spark.read.csv('/home/rulicering/Datos_Proyecto_Ozono/Procesado/Dato_Final/Datos.csv',inferSchema= True,header=True)
df_clima_prediccion = spark.read.csv('/home/rulicering/Datos_Proyecto_Ozono/Procesado/Clima/Clima_Prediccion-hoy.csv',inferSchema= True,header=True)
df_calendario = spark.read.csv('/home/rulicering/Datos_Proyecto_Ozono/Procesado/Calendario/Calendario_2013-2020.csv',inferSchema= True,header=True)

In [7]:
df_datos = df_datos.drop("_c0")
df_clima_prediccion = df_clima_prediccion.drop("_c0")
df_calendario = df_calendario.drop("_c0")

In [8]:
magnitudes= df_datos.columns[8:]
magnitudes_clima = df_datos.columns[-5:]
magnitudes_aire = df_datos.columns[8:-5]

In [9]:
dic_clima = { "VIENTO":"81",
                 "DIRECCION": "82",
                 "TEMPERATURA": "83",
                 "PRESION": "87",
                 "LLUVIA":"89"
}

## [1.1] - Datos para prediccion - Prediccion clima + Calendario + Estaciones

In [10]:
ayer = (datetime.date.today() + datetime.timedelta(days = -1)).strftime("%Y%m%d")
hoy = datetime.date.today().strftime("%Y%m%d")

In [11]:
df_estaciones_aire = df_datos.filter(df_datos["FECHA"]== ayer).select("CODIGO_CORTO")

In [12]:
cod_estaciones_aire = [elem[0] for elem in df_estaciones_aire.collect()]

In [13]:
cod_estaciones_aire.sort()

In [14]:
df_hoy = df_calendario.filter(df_calendario["FECHA"]== hoy)

In [15]:
#Calendario + Prediccion clima
df_clima_hoy = df_hoy.join(df_clima_prediccion,on= "FECHA")

In [16]:
#Estaciones cross datos clima y calendario
df_datos_hoy = df_estaciones_aire.crossJoin(df_clima_hoy)

In [17]:
cols = df_datos_hoy.columns
cols = cols[0:1] + cols[5:8]+ cols[1:5]+ cols[8:]

In [18]:
df_datos_hoy = df_datos_hoy.select(cols)

### [1.1.0] - Probabilidad lluvia -> Prediccion lluvia m/l2
    Si la probabilidad es > 50%:
        se hace la media por estacion del historial de precipitaciones
        cogiendo datos de +-10 días al dia de hoy de cada año anterior

In [41]:
def probabilidad_a_lluvia_y_presion_ayer(df_datos,df_datos_hoy):
    mes_dia_min = (datetime.date.today() +  datetime.timedelta(days = -10)).strftime("%m%d")
    mes_dia_max = (datetime.date.today() +  datetime.timedelta(days = 10)).strftime("%m%d")
    df_historial = df_datos.filter((df_datos["FECHA"]%1000 >= mes_dia_min) & (df_datos["FECHA"]%1000 <= mes_dia_max))
    df_datos_hoy = df_datos_hoy.drop('%' + dic_clima["LLUVIA"])
    l_df = []
    for estacion in cod_estaciones_aire:
        df_datos_hoy_estacion = df_datos_hoy.filter(df_datos_hoy["CODIGO_CORTO"]==estacion)
        aux = df_historial.filter(df_historial["CODIGO_CORTO"]== estacion).select("FECHA",dic_clima["PRESION"],dic_clima["LLUVIA"]).na.drop()
        #Precipitacions
        prob_lluvia_hoy = df_clima_prediccion.select("%" +dic_clima["LLUVIA"]).collect()[0][0]
        prec = 0
        if(float(prob_lluvia_hoy) > 50):
            try:
                prec = aux.select(dic_clima["LLUVIA"]).groupBy().mean().collect()[0][0]
            except:
                print("[WARN]: No hay lluvias historicas en ese rango de fechas")
        df_datos_hoy_estacion = df_datos_hoy_estacion.withColumn(dic_clima["LLUVIA"],F.lit(prec))
            
        #Presion
        ayer = (datetime.date.today() + datetime.timedelta(days= -1)).strftime("%Y%m%d")
        presion_ayer = aux.filter(aux["FECHA"]==ayer).select(dic_clima["PRESION"]).collect()[0][0]
        df_datos_hoy_estacion = df_datos_hoy_estacion.withColumn(dic_clima["PRESION"],F.lit(presion_ayer))
        
        l_df.append(df_datos_hoy_estacion)
        
    df_datos_hoy = l_df[0]
    for i in range(1,len(l_df)):
        df_datos_hoy = df_datos_hoy.union(l_df[i])
    return df_datos_hoy

In [24]:
df_datos_hoy = probabilidad_a_lluvia_y_presion_ayer(df_datos,df_datos_hoy)


## [1.2] - Union Datos + Datos hoy

In [270]:
#df_datos_y_hoy= df_datos.union(df_datos_hoy)

## [1.1] - Dar cada fila de datos + datos clima y contaminacion ayer

In [39]:
ventana = Window.partitionBy("CODIGO_CORTO").orderBy("FECHA")

In [42]:
for magnitud in magnitudes:
    df_datos = df_datos.withColumn("A_%s"%magnitud, F.lag(magnitud,1,None).over(ventana))

In [43]:
df_datos.columns

['CODIGO_CORTO',
 'ANO',
 'MES',
 'DIA',
 'FECHA',
 'DIASEMANA',
 'TIPODIA',
 'CONFINAMIENTO',
 '1',
 '6',
 '7',
 '8',
 '9',
 '10',
 '12',
 '14',
 '20',
 '30',
 '35',
 '42',
 '43',
 '44',
 '81',
 '82',
 '83',
 '87',
 '89',
 'A_1',
 'A_6',
 'A_7',
 'A_8',
 'A_9',
 'A_10',
 'A_12',
 'A_14',
 'A_20',
 'A_30',
 'A_35',
 'A_42',
 'A_43',
 'A_44',
 'A_81',
 'A_82',
 'A_83',
 'A_87',
 'A_89']

In [24]:
df_datos.withColumn("A_81",F.lag("81",1,None).over(ventana)).show()

+------------+----+---+---+--------+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+---+----+------------------+------+---+---------+-------+-------------+----+
|CODIGO_CORTO| ANO|MES|DIA|   FECHA|   1|   6|   7|   8|   9|  10|   12|  14|  20|  30|  35|  42|  43|  44| 81|  82|                83|    87| 89|DIASEMANA|TIPODIA|CONFINAMIENTO|A_81|
+------------+----+---+---+--------+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+---+----+------------------+------+---+---------+-------+-------------+----+
|          27|2014|  1|  1|20140101|null|null| 4.0|27.0|null|null| 32.0|34.0|null|null|null|1.37|1.18|0.19|3.6|22.0|              8.45| 950.0|5.5|        3|      0|            0|null|
|          27|2014|  1|  2|20140102|null|null| 5.0|29.0|null|null| 37.0|37.0|null|null|null|1.37| 1.2|0.17|5.8|20.0|              11.0|947.35|5.4|        4|      1|            0| 3.6|
|          27|2014|  1|  3|20140103|null|null| 7.0|30.0|null|null| 42.0|39.0|nul

In [91]:
#https://stackoverflow.com/questions/45011320/how-to-get-data-of-previous-row-in-apache-spark

In [55]:
df_datos.columns

['CODIGO_CORTO',
 'ANO',
 'MES',
 'DIA',
 'FECHA',
 '1',
 '6',
 '7',
 '8',
 '9',
 '10',
 '12',
 '14',
 '20',
 '30',
 '35',
 '42',
 '43',
 '44',
 '81',
 '82',
 '83',
 '87',
 '89',
 'DIASEMANA',
 'TIPODIA',
 'CONFINAMIENTO']

## [1.1] - Estaciones aire + prediccion clima + calendario

In [46]:
magnitudes_clima = df_clima_prediccion.columns[4:]

In [47]:
ayer = (datetime.date.today() - datetime.timedelta(days=-2)).strftime("%Y%m%d")

In [52]:
df_datos_ayer = df_datos.filter(df_datos["FECHA"]==20200101)

In [54]:
df_datos_ayer = df_datos_ayer.drop("ANO","MES","DIA","FECHA","DIASEMANA","TIPO_DIA","CONFINAMIENTO")

In [ ]:
df_prediccion

In [ ]:
for magnitud in magnitudes_clima:
    df_datos_ayer = df_datos_ayer.withColumn()

## [1.1] - Paso a Pandas

In [4]:
pd_aire = df_aire.drop("_c0").toPandas()
pd_clima = df_clima.drop("_c0").toPandas()
pd_estaciones = df_estaciones.drop("_c0").toPandas()
pd_calendario = df_calendario.drop("_c0").toPandas()

## [1.2] - Tipos
    
    CODIGO_CORTO -> STRING/OBJECT
    ANO -> INTEGER
    MES -> INTEGER
    DIA -> INTEGER
    FECHA -> INTEGER
    MEDICIONES -> DOUBLE

### [1.2.0] -  Estaciones

In [5]:
#None to NULO
pd_estaciones = pd_estaciones.replace(('None',None),np.nan)

In [6]:
#Columnas estaciones medicion - 2014
regex = reg.compile("E_AEMET")
c_grupo_14 = [elem for elem in list(filter(regex.search,df_estaciones.columns))]
c_grupo_14

['E_AEMET']

In [139]:
#Columnas estaciones medicion - 2019
regex = reg.compile("E_\d\d")
c_estacionxmagnitud_19 = [elem for elem in list(filter(regex.search,df_estaciones.columns))]
c_estacionxmagnitud_19

['E_81', 'E_82', 'E_83', 'E_87', 'E_89']

In [135]:
#Columnas estaciones medicion a string
for columna in c_estacionxmagnitud_19:
    pd_estaciones[columna] = pd_estaciones[columna].astype(str)

In [136]:
#pd_estaciones.dtypes

###  [1.2.1] - Aire

In [ ]:
#None to NULO
pd_aire = pd_aire.replace(('None',None),np.nan)

In [123]:
#CODIGOS CORTOS A Strig
pd_aire["CODIGO_CORTO"] = pd_aire["CODIGO_CORTO"].astype(str)

In [168]:
pd_aire = pd_aire.sort_values(by="FECHA")

In [124]:
#pd_aire.dtypes

### [1.2.2] - Clima

In [131]:
#None to NULO
pd_clima = pd_clima.replace(('None',None),np.nan)

In [125]:
#Listar las columnas que miden magnitudes
columnas_valoresmagnitudes = list(pd_clima.columns)[5:]

In [127]:
#Columnas magnitudes a float
for columna in columnas_valoresmagnitudes:
    #Comas por puntos
    pd_clima[columna]  =  [reg.sub(',','.',str(x)) for x in pd_clima[columna]]
    # String to float
    pd_clima[columna] = pd_clima[columna].astype(float)

In [169]:
pd_clima = pd_clima.sort_values(by="FECHA")

In [129]:
#pd_clima.dtypes

### [1.2.3] - Calendario

In [132]:
pd_calendario = pd_calendario.replace(('None',None),np.nan)

## [1.3] - Darle grupo a estaciones de AIRE

    2014-2018 -> E_AEMET
    
    2019-NOW -> 81,82,83,87,89

In [141]:
#Sacamos estaciones de aire y codigos de estaciones de clima asociadas
pd_estaciones_aire = pd_estaciones[pd_estaciones["MIDE_AIRE"]>0]

In [142]:
#Nos quedamos con las columnas que queremos
c_agrupamiento = ["CODIGO_CORTO"] + c_grupo_14 + c_estacionxmagnitud_19
pd_estaciones_aire = pd_estaciones_aire[c_agrupamiento]

In [143]:
#Unimos ambos datasets
pd_aire = pd_aire.merge(pd_estaciones_aire, on =["CODIGO_CORTO"])

# [2] -  UNIR AIRE + CLIMA

## [2.0] - Separamos datos contaminacion (2014-18 | 2019-NOW)

In [232]:
#Datos 14-18
pd_datos_14_18 = pd_aire[pd_aire["FECHA"]<20190101]

In [233]:
#Datos 19-NOW
pd_datos_19_NOW = pd_aire[pd_aire["FECHA"]>=20190101]

## [2.1] - MERGE (ESTACION AIRE, DIA) + (DATOS CLIMA )

In [237]:
# Separamos columnas de info (comunes) y las columnas de datos
columnas = list(pd_clima.columns) 
c_info = columnas[:5]
c_magnitudes = columnas[5:]

In [238]:
#Asociamos cada columna de datos de clima (magnitud) a la estación de aire correspondiente.  
for magnitud in c_magnitudes:
    cols = c_info.copy()
    cols.append(magnitud)
    
    pd_clima_magnitud = pd_clima[cols]

    #2014-2018
    pd_clima_magnitud_14_18 = pd_clima_magnitud.rename(columns={"CODIGO_CORTO":c_grupo_14[0]})
    pd_datos_14_18 = pd_datos_14_18.merge(pd_clima_magnitud_14_18,on = ["ANO", "MES", "DIA","FECHA",c_grupo_14[0]])
    
    #2019-NOW
    pd_clima_magnitud_19_NOW = pd_clima_magnitud.rename(columns={"CODIGO_CORTO":"E_%s"%magnitud})
    pd_datos_19_NOW = pd_datos_19_NOW.merge(pd_clima_magnitud_19_NOW,on = ["ANO", "MES", "DIA","FECHA","E_%s"%magnitud])

## [2.2] - Unimos partes generadas por el MERGE

In [242]:
#pd_datos_14_18.dtypes

In [243]:
#pd_datos_19_NOW.dtypes

In [252]:
pd_datos = pd.concat([pd_datos_14_18,pd_datos_19_NOW])

# [3] - Incluir CALENDARIO

In [273]:
pd_datos_y_calendario = pd_datos.merge(pd_calendario, on = "FECHA")

In [275]:
#pd_datos_y_calendario.dtypes

# [4] - FORMATO

In [276]:
#Borramos las columnas utilizadas para relacionar estaciones de aire y clima
a_borrar = c_grupo_14 + c_estacionxmagnitud_19

In [277]:
pd_final = pd_datos_y_calendario.drop(columns = a_borrar)

# [5] - EXPORTAR

In [ ]:
#Versiones
hoy = datetime.date.today().strftime("%Y-%m-%d")
pd_final.to_csv("/home/rulicering/Datos_Proyecto_Ozono/Procesado/Dato_Final/Datos_2014-NOW-" + hoy + ".csv")

In [278]:
pd_final.to_csv("/home/rulicering/Datos_Proyecto_Ozono/Procesado/Dato_Final/Datos_2014-NOW.csv")

# [EXTRA] - CHECKEO

In [262]:
#pd_chequeo = pd_datos[['CODIGO_CORTO', 'FECHA','E_AEMET','E_81', 'E_82', 'E_83', 'E_87', 'E_89', '81', '82', '83', '87', '89']]

In [261]:
#2014-2018
#pd_chequeo[(pd_datos["FECHA"]==20140101)].sort_values(by="E_AEMET")
#pd_clima[pd_clima["FECHA"]==20140101]

#2019-NOW
#pd_chequeo[(pd_datos["FECHA"]==20190101)]
#pd_clima[pd_clima["FECHA"]==20190101]